# Chapter 4 word2vec 속도 개선

CBOW는 말뭉치의 어휘 수가 많아지면 계산량이 커져 계산 시간이 오래 걸리는 단점이 존재한다.

속도 개선 방법으로 Embedding, 네거티브 샘플링을 이용한다.

## 4.1 word2vec 개선 1

#### CBOW 병목 지점
- 입력층의 원핫 표현과 가중치 행렬의 곱 계산
    - 원핫 벡터의 문제로 Embedding 계층 도입하면 해결
- 은닉층과 가중치 행렬의 곱 및 Softmax 계산
    - 네거티브 샘플링이라는 새로운 손실함수를 도입하면 해결

### 4.1.1 Embedding 계층

원핫 벡터와 가중치의 곱이라는 거대한 계산은 결국 가중치 행렬의 특정 행을 뽑아내는 것뿐이다. 즉, 곱하지 않고 단순히 단어 ID에 해당하는 행을 뽑아내는 계층으로 표현할 수 있다. 이를 Embedding 계층이라고 한다. Embedding 계층에는 단어 임베딩(분산 표현)을 저장한다.

### 4.1.2 Embedding 계층 구현

In [1]:
import numpy as np
W = np.arange(21).reshape(7, 3)
W

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [2]:
W[2]

array([6, 7, 8])

In [3]:
W[5]

array([15, 16, 17])

In [4]:
idx = np.array([1, 0, 3, 0])
W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

In [5]:
class Embedding:
    
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
        
        
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        
        return out
    
    
    def backward(self, dout):
        dW, = self.grads
        dw[...] = 0
        
        # 중복 문제를 해결하기 위해 더하기를 해준다.
        np.add.at(dW, self.idx, dout)
        # 혹은
        # for i, word_id in enumerate(self.idx):
        #     dW[word_id] += dout[i]
        # for문으로도 구현할 수 있지만 numpy를 사용하는 것이 더욱 효율적
        
        return None

backward의 경우에는 단순히 전해진 기울기를 흘려보내기만 한다. 단 추출된 행으로 흘려보낸다. 단순히 할당하면 동일한 idx를 가리키면 먼저 가리킨 값은 덮어씌워져서 사라진다. 이를 해결하기 위해 더하기를 한다.

## 4.2 word2vec 개선 2

**네거티브 샘플링**을 통해서 행렬 곱과 Softmax 계층의 계산에서 발생하는 병목을 해소할 수 있다.

### 4.2.1 은닉층 이후 계산의 문제점

#### 병목 발생 지점
1. 은닉층의 뉴런과 가중치 행렬의 곱
2. Softmax 계층의 계산

행렬 곱을 가볍게 하고 Softmax를 대체할 가벼운 연산이 필요하다.

### 4.2.2 다중 분류에서 이진 분류로

다중 분류: 맥락(you, goodbye)일 때 타깃 단어는 무엇인가?  
이진 분류: 맥락(you, goodbye)일 때 타깃 단어(say)인가?

#### CBOW에선?

출력 측의 가중치 행렬에서 필요한 타깃에 해당되는 열 벡터만 뽑아서 은닉층 뉴런과 내적한다. 이러면 출력 층 뉴런은 1개만 필요하다. 이렇게 출력된 점수는 시그모이드 함수를 통해 확률로 변환된다.

<img src="./images/fig 4-7.png" width=600>
<img src="./images/fig 4-8.png" width=600>

### 4.2.3 시그모이드 함수와 교차 엔트로피 오차

이진 분류에서 시그모이드로 확률 값을 얻어내고 손실 함수로 교차 엔트로피 오차를 이용한다. 시그모이드 + 교차 엔트로피 오차 계층의 역전파는 $y-t$, 즉 정답 레이블과 출력한 확률의 차이다.

교차 엔트로피 오차를 조합한 역전파는 모두 y-t의 결과로 나타난다.

### 4.2.4 다중 분류에서 이진 분류로 (구현)

<img src="./images/fig 4-12.png" width=600>
<img src="./images/fig 4-13.png" width=600>

- 입력층의 embedding 계층으로 연산을 단순화
- 출력층의 embedding dot 계층으로 연산 단순화

In [6]:
class EmbeddingDot:
    
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
        
    
    def forward(self, h, idx):   # idx는 배열, 미니배치 처리를 가정
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)
        
        self.cache = (h, target_W)
        
        return out
    
    
    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)
        
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        
        return dh

### 4.2.5 네거티브 샘플링

다중 분류 문제를 이진 분류로 다루려면 정답과 오답 각각에 대해 바르게 분류할 수 있어야 한다.

네거티브 샘플링은 모든 부정적인 예를 대상으로 하지 않고, 근사적인 해법으로 여러 부정적인 예 몇 개를 선택한다. 긍정적인 예의 손실과 샘플링된 부정적 예의 손실을 더해서 최종 손실을 구한다. 긍정적인 예는 정답 레이블로 1을 부정적인 예는 정답 레이블로 0을 입력한다.

<img src="./images/fig 4-17.png" width=600>

### 4.2.6 네거티브 샘플링의 샘플링 기법

부정적인 예를 샘플링하는 방법은 통계를 기초로 한다. 단어의 출현 횟수를 확률분포로 나타내고 이 분포를 이용해서 단어를 샘플링한다. 드문 단어를 잘 처리하는 일의 중요도는 낮기 때문에 이 방법을 사용한다.

In [7]:
import numpy as np

np.random.choice(10)

0

In [8]:
words = ['you', 'say', 'goodbye', 'I', 'hello', '.']
np.random.choice(words)

'you'

In [9]:
np.random.choice(words, size=5)

array(['.', 'say', 'I', '.', 'say'], dtype='<U7')

In [10]:
np.random.choice(words, size=5, replace=False)

array(['say', 'I', 'you', 'hello', '.'], dtype='<U7')

In [11]:
# 확률 분포에 따라 샘플링
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
np.random.choice(words, p=p)

'you'

다만, 0.75 제곱을 해서 낮은 확률의 단어를 조금 더 많이 뽑게 조절해준다(확률의 총합은 1이 돼야 하기 때문에 수정 후 확률분포의 총합으로 나눠준다.).

In [12]:
p = [0.7, 0.29, 0.01]
new_p = np.power(p, 0.75)
new_p /= np.sum(new_p)
print(new_p)

[0.64196878 0.33150408 0.02652714]


In [13]:
from negative_sampling_layer import UnigramSampler

In [14]:
corpus = np.array([0, 1, 2, 3, 4, 1, 2, 3])
power = 0.75
sample_size = 2

sampler = UnigramSampler(corpus, power, 3)
target = np.array([1, 3, 0])
negative_sample = sampler.get_negative_sample(target)
print(negative_sample)

[[3 4 2]
 [0 4 2]
 [3 2 4]]


### 4.2.7 네거티브 샘플링 구현

In [15]:
class NegativeSamplingLoss:
    
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size+1)] # 긍정적 예를 다루는 계층이 하나 더 필요해서
        self.embed_dot_layers  = [EmbeddingDot(W) for _ in range(sample_size+1)]
        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads
            
            
    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)
        
        # 긍정적 예 순전파
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)
        
        # 부정적 예 순전파
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]
            score  = self.embed_dot_layers[1 + i].forward(h, negative_target)
            loss += self.loss_layers[1 + i].forward(score, negative_label)
            
        return loss
    
    
    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layer, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)
            
        return dh

## 4.3 개선판 word2vec 학습

PTB 데이터셋을 사용해서 실습

### 4.3.1 CBOW 모델 구현

In [16]:
import numpy as np
from common.layers import Embedding
from negative_sampling_layer import NegativeSamplingLoss

In [17]:
class CBOW:
    
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size
        
        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')    # 형상 주의: 출력층에서 embedding 있어서 입력층과 동일한 형상
        
        # 계층 생성
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)
        
        # 모든 가중치와 기울기를 배열에 모은다.
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads
            
        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in
        
        
    # 맥락과 타깃은 원핫 벡터가 아닌 단어 ID이다. 
    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss
    
    
    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
            
        return None

### 4.3.2 CBOW 모델 학습 코드

In [1]:
import numpy as np
from common import config

config.GPU = True
# GPU 버전 구동 안됨

import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from cbow import CBOW
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb

AttributeError: 'cupy.core._kernel.ufunc' object has no attribute 'at'

In [ ]:
# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)
    
# 모델 등 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)
    
# 너무 오래 걸려서 중단

### 4.3.3 CBOW 모델 평가

In [ ]:
from common.util import most_similar
import pickle

In [ ]:
pkl_file = 'cbow_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']
    
querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)

In [ ]:
from common.util import analogy

In [ ]:
analogy('king', 'man', 'queen', word_to_id, id_to_word, word_vecs, top=5)
analogy('take', 'took', 'go', word_to_id, id_to_word, word_vecs, top=5)
analogy('car', 'cars', 'child', word_to_id, id_to_word, word_vecs, top=5)
analogy('good', 'better', 'bad', word_to_id, id_to_word, word_vecs, top=5)

CBOW 모델은 괜찮은 특성을 학습한 분산 표현을 얻을 수 있을 뿐만 아니라, 덧셈, 뺄셈을 통한 유추 연산을 할 수 있다. 하지만 PTB 데이터셋도 실세계의 모든 단어를 포함할 정도로 말뭉치가 크지 않기 때문에 PTB를 통해 얻은 분산 표현으로 모든 유추 문제를 풀기에는 한계가 있다.

## 4.4 word2vec 남은 주제

### 4.4.1 word2vec을 사용한 애플리케이션의 예

전이학습을 할 수 있다. 전이학습이란 한 분야에서 학습된 모델을 이용해서 다른 분야에 적용하는 것이다. 텍스트 분류, 문서 클러스터링, 감정 분석 등 자연어 처리 작업을 할 때 분산 표현을 만드는 것부터 학습하진 않는다. 이미 학습된 분산 표현을 얻어와서 사용한다.

분산표현은 단어를 고정 길이의 벡터로 표현할 수 있다. 이를 통해 문장도 고정된 벡터로 변환할 수 있는데 이를 bag-of-words라고 한다. RNN을 이용해서 더욱 세련된 방법으로 고정된 길이의 벡터로 표현할 수도 있다.

<img src="./images/fig 4-21.png" width=600>

고정 벡터로 변환할 수 있다면 머신러닝 시스템의 입력으로 사용할 수 있다. 다만 학습 데이터가 굉장히 많다면 처음부터 학습해도 된다. 전이학습을 잘 이용하면 속도와 정확도 모두 높일 수 있다.

### 4.4.2 단어 벡터 평가 방법

단어의 분산 표현의 정확도를 측정할 땐 시스템과 분리해서 평가한다.

검증 방법
- 사람이 작성한 단어 유사도를 검증 세트로 사용해서 평가 (코사인 유사도)
- 유추 문제를 활용한 평가, 유추 문제를 만들고 정답률로 측정
    - 단어의 의미나 문법적인 문제를 제대로 이해하고 있는지 측정 가능
    
분산 표현이 우수하다고 어플리케이션의 성능도 좋은 것은 아니다.

일반적인 경우
- 모델에 따라 정확도가 다르다
- 말뭉치가 클수록 결과가 좋다.
- 벡터 차원 수는 너무 커지면 정확도가 나빠진다.